In [14]:
from datetime import datetime as dt,timedelta
import xarray as xr
import cfgrib




In [2]:
def _get_colIrr_ds(self,filename):

    # colIrr:
    # (+) Down SW rad flux at nominal top (dswrf)
    # (-) Down SW rad flux at surface (dswrf)
    # (-) Down LW rad flux at surface (dlwrf)
    # (-) Up SW rad flux at nominal top (uswrf)
    # (+) Up SW rad flux at surface (uswrf)
    # (-) Up LW rad flux at nominal top (ulwrf)
    # (+) Up LW rad flux at surface (ulwrf)
    # (+) Up sensible heat flux at surface (shf)
    # (+L*) Precipitation ()
    # 
    # f'fcst_phy2m125/{time-timedelta(days=1):%Y%m}/fcst_phy2m125.{time-timedelta(days=1):%Y%m%d%H}'
    #

    ds = xr.open_dataset(filename, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys':{'typeOfLevel': 'surface'}})

    dsw_sfc = ds['dswrf'] # W m**-2
    dlw_sfc = ds['dlwrf'] # W m**-2
    usw_sfc = ds['uswrf'] # W m**-2
    ulw_sfc = ds['ulwrf'] # W m**-2
    shf = ds['shf'] # W m**-2
    pcp = ds['tpratsfc'] # mm per day

    densw = 1e3 # kg m**-3
    day_per_sec = 1/(60*60*24)
    m_per_mm = 1e-3
    pcp = pcp * day_per_sec * m_per_mm * densw # kg m**-2 s**-1
    L = 2.260e6 # J kg**-1

    ds = xr.open_dataset(filename, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys':{'typeOfLevel': 'nominalTop'}})

    dsw_nt = ds['dswrf'] # W m**-2
    usw_nt = ds['uswrf'] # W m**-2
    ulw_nt = ds['ulwrf'] # W m**-2

    colIrr = dsw_nt - dsw_sfc - dlw_sfc - usw_nt + usw_sfc - ulw_nt + ulw_sfc + shf + L*pcp

    return colIrr.to_dataset(name='colIrr')

In [7]:
ts = dt(2023,12,9)
tl = dt(2023,12,31)

In [19]:
print(filename)
# Open the grib file and print the index
with cfgrib.open_file(filename,filter_by_keys={'typeOfLevel': 'isobaricInPa'}) as f:
    print(f.index)

/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_p125_hgt.2023120900


AttributeError: __enter__

In [71]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_p125_hgt.2023120900'
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'})
# ds = xr.open_dataset(filename, engine='cfgrib')
ds

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 40, latitude: 145, longitude: 288)
Coordinates:
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * latitude       (latitude) float64 90.0 88.75 87.5 ... -87.5 -88.75 -90.0
  * longitude      (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    valid_time     datetime64[ns] ...
Data variables:
    gh             (isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T13:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [90]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_land125.2023120100'
# ds = xr.open_dataset(filename, engine='cfgrib',backend_kwargs={'filter_by_keys':{'cfVarName':'ussl'}})
# ds = xr.open_dataset(filename, engine='cfgrib')
ds = xr.open_dataset(filename, engine='cfgrib',backend_kwargs={'filter_by_keys':{'cfVarName':'liqvsm'}})
print(ds)

ds = ds.isel(depthBelowLandLayer=slice(0,7)).mean(dim='depthBelowLandLayer')
print(ds)

<xarray.Dataset>
Dimensions:              (depthBelowLandLayer: 7, latitude: 145, longitude: 288)
Coordinates:
    time                 datetime64[ns] ...
    step                 timedelta64[ns] ...
  * depthBelowLandLayer  (depthBelowLandLayer) float64 0.0 0.02 ... 0.99 1.99
  * latitude             (latitude) float64 90.0 88.75 87.5 ... -88.75 -90.0
  * longitude            (longitude) float64 0.0 1.25 2.5 ... 356.2 357.5 358.8
    valid_time           datetime64[ns] ...
Data variables:
    liqvsm               (depthBelowLandLayer, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-17T11:59 GRIB to CDM+CF via cfgrib-0.9.1...
<xarray.Dataset>
Dimensions:              (depthBelowLandLayer

In [42]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/fcst_phy2m125.2023120900'
ds = xr.open_dataset(filename, engine='cfgrib', \
                        backend_kwargs={'filter_by_keys':{'typeOfLevel': 'surface'}})

dsw_sfc = ds['dswrf'] # W m**-2
dlw_sfc = ds['dlwrf'] # W m**-2
usw_sfc = ds['uswrf'] # W m**-2
ulw_sfc = ds['ulwrf'] # W m**-2
# shf = ds['shf'] # W m**-2
# pcp = ds['tpratsfc'] # mm per day, is this true? It is, so the unit has to change
pcp = ds1['tprate'] # new name kg m-2 s-1
shf = ds['ishf']


In [43]:
ds = xr.open_dataset(filename, engine='cfgrib', \
                        backend_kwargs={'filter_by_keys':{'typeOfLevel': 'nominalTop'}})

dsw_nt = ds['dswrf'] # W m**-2
usw_nt = ds['uswrf'] # W m**-2
ulw_nt = ds['ulwrf'] # W m**-2


In [51]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_surf125.2023110100'
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea'})
ds

<xarray.Dataset>
Dimensions:     (latitude: 145, longitude: 288)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    meanSea     float64 ...
  * latitude    (latitude) float64 90.0 88.75 87.5 86.25 ... -87.5 -88.75 -90.0
  * longitude   (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    valid_time  datetime64[ns] ...
Data variables:
    prmsl       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T12:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [57]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_surf125.2023110100'
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'heightAboveGround'})
ds

skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 676, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 607, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 676, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 607, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and

<xarray.Dataset>
Dimensions:            (latitude: 145, longitude: 288)
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 88.75 87.5 ... -87.5 -88.75 -90.0
  * longitude          (longitude) float64 0.0 1.25 2.5 ... 356.2 357.5 358.8
    valid_time         datetime64[ns] ...
Data variables:
    t2m                (latitude, longitude) float32 ...
    unknown            (latitude, longitude) float32 ...
    sh2                (latitude, longitude) float32 ...
    r2                 (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T12:39 GRIB to CDM+CF via cfgrib-0.9.1...

In [68]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/fcst_surf125.2023120900'
# ds = xr.open_dataset(filename, engine='cfgrib')
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'heightAboveGround'})
ds

skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 676, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 607, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 676, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ycheng/.conda/envs/cpc/lib/python3.10/site-packages/cfgrib/dataset.py", line 607, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and

<xarray.Dataset>
Dimensions:            (latitude: 145, longitude: 288)
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 88.75 87.5 ... -87.5 -88.75 -90.0
  * longitude          (longitude) float64 0.0 1.25 2.5 ... 356.2 357.5 358.8
    valid_time         datetime64[ns] ...
Data variables:
    t2m                (latitude, longitude) float32 ...
    unknown            (latitude, longitude) float32 ...
    sh2                (latitude, longitude) float32 ...
    r2                 (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T12:54 GRIB to CDM+CF via cfgrib-0.9.1...

In [70]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/bnd_ocean125.2023110100'
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'surface'})
# ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'heightAboveGround'})
ds


<xarray.Dataset>
Dimensions:     (latitude: 145, longitude: 288)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 90.0 88.75 87.5 86.25 ... -87.5 -88.75 -90.0
  * longitude   (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    valid_time  datetime64[ns] ...
Data variables:
    siconc      (latitude, longitude) float32 ...
    unknown     (latitude, longitude) float32 ...
    sst         (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T12:58 GRIB to CDM+CF via cfgrib-0.9.1...

In [72]:
filename = '/Projects/jalbers_process/CPC_LIM/yuan_ming/JRA-3Q/anl_p125_strm.2023120900'
ds = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'})


In [73]:
ds

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 40, latitude: 145, longitude: 288)
Coordinates:
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * latitude       (latitude) float64 90.0 88.75 87.5 ... -87.5 -88.75 -90.0
  * longitude      (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    valid_time     datetime64[ns] ...
Data variables:
    strf           (isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             rjtd
    GRIB_centreDescription:  Japanese Meteorological Agency - Tokyo
    GRIB_subCentre:          241
    Conventions:             CF-1.7
    institution:             Japanese Meteorological Agency - Tokyo
    history:                 2024-01-13T17:54 GRIB to CDM+CF via cfgrib-0.9.1...